# Sumário

- [Introdução](#Introdu%C3%A7%C3%A3o)
- [Métodos de Filtragem (Filter Methods)](#M%C3%A9todos-de-Filtragem-%28Filter-Methods%29)
    - [Correlação de Pearson](#Correla%C3%A7%C3%A3o-de-Pearson)
    - [Mutual Information](#Mutual-Information)
    - [Maximal Information Coefficient](#Maximal-Information-Coefficient)
- [Métodos de Empacotamente (Wrapper Methods)](#M%C3%A9todos-de-Empacotamento-%28Wrapper-Methods%29)
    - [Stability Selection](#Stability-Selection)
    - [Eliminação recursiva de atributos](#Elimina%C3%A7%C3%A3o-recursiva-de-atributos)
    - [Boruta](#Boruta)
- [Diferença entre métodos de filtragem e empacotamento](#Diferen%C3%A7a-entre-m%C3%A9todos-de-filtragem-e-empacotamento)
- [Métodos Embarcados (Embedded Methods)](#M%C3%A9todos-Embarcados-%28Embedded-Methods%29)
   - [Linear Regression](#Linear-Regression)
   - [Modelos Reguladores](#Modelos-Reguladores)
       - [L1 regularization/Lasso](#L1-regularization-/-Lasso)
       - [L2 regularization/Ridge](#L2-regularization-/-Ridge)
   - [Random Forest](#Random-Forest)
       - [Mean Decrease Impurity](#Mean-Decrease-Impurity)
       - [Mean Decrease Accuracy](#Mean-Decrease-Accuracy)
- [Exemplo prático](#Exemplo-pr%C3%A1tico)
- [Conclusões](#Conclus%C3%B5es)
- [Referências](#Refer%C3%AAncias)

# Imports and Settings

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.datasets import make_regression, load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE, RFECV, f_regression
from sklearn.linear_model import LinearRegression, Lasso, Ridge, RandomizedLasso
from sklearn.metrics import r2_score
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from minepy import MINE

# Introdução

Uma das coisas mais importantes no Machine Learning é a __engenharia de atributos__, que envolve tanto a parte de __seleção de atributos__ (_feature selection_) quanto __criação de atributos__ (_feature creation_). Neste notebook, nós vamos focar na seleção de atributos (features).

A maioria dos algoritmos de machine learning sofre com um problema: se você der atributos ruins para ele, o resultado provavelmente também será ruim. Atributos ruins involvem desde dados inconsistentes ou errados até ruído nos dados. Isso é ainda mais crucial quando o número de atributos é muito grande. Em geral, você não precisa usar todos os atributos. Além de reduzir o tempo de treinamento, menos atributos também significam menos problemas para se preocupar. É como aquele velho diatado diz: _"Menos é mais!"_

A seleção de atributos é importante, pois:

- deixa o __treinamento mais rápido__, diminuindo o tempo de treinamento;
- __reduz a complexidade de um modelo__ e torna-o mais simples de interpretar;
- __facilita o entendimento da relação dos atributos com a saída__;
- __melhora a precisão do modelo__ se o subconjunto "ótimo" for escolhido; e
- __reduz a chance de overfitting e melhora a generalização__

Podemos agrupar os algoritmos de seleção de atributos em 3 tipos: __métodos de filtragem__ (_filter methods_), __métodos de empacotamento__ (_wrapper methods_) e __métodos embarcados__ (_embedded methods_). Nas seções a seguir, veremos as diferenças entre eles.

# Métodos de Filtragem (Filter Methods)

<img src='images/selecao_filter_methods.png'>

Métodos de filtragem, também conhecidos como __seleção de atributos univariada__ (_univariate feature selection_), são geralmente utilizados como pré-processamento. A seleção dos atributos é independente de qualquer algoritmo de machine learning. Ao invés disso, __cada atributo é examinado individualmente para determinar a força do seu relacionamento com a variável resposta (saída). Esse métodos são simples de rodar, entender e, em geral, são bons para se obter um bom entendimento dos dados__ (mas não necessariamente otimizar o conjunto de atributos para melhor generalização). Existem um monte de métodos de filtragem. Como guia básico, você pode considerar a seguinte tabela que define alguns dos métodos de seleção univariada:

| Atributo\Resposta |        Contínuo       | Categórico            |
|:-----------------:|:---------------------:|:---------------------:|
|   __Contínuo__    | Correlação de Pearson | LDA                   |
|  __Categórico__   |         ANOVA         | Chi-Square ($\chi^2$) |

- __Correlação de Pearson__: usada para quantificar a correlação linear entre duas variáveis contínuas $X$ e $Y$. A saída varia de $[-1, +1]$, onde $-1$ significa correlação negativa perfeita (quando $X$ cresce, $Y$ diminui) e $+1$ significa correlação positiva perfeita (quando $X$ cresce, $Y$ cresce). O coeficiente de correlação de Pearson $P(X,Y)$ é dado por:

$$ P(X,Y) = \frac{cov(X, Y)}{\sigma_X \sigma_Y} $$

- __LDA__: significa __análise discriminante linear__ (_linear discriminant analysis_) e é usada para encontrar o melhor combinação linear de atributos que separam duas ou mais classes. Você pode conferir um notebook completo sobre LDA neste mesmo repositório clicando [aqui](LDA.ipynb).
- __ANOVA__: teste estatístico que significa __análise da variância__ (_analysis of variance_). É similar ao LDA, porém funcioa para um ou mais atributos categóricos independentes e um atributo contínuo como saída. Esse teste indica se a média de diversos grupos são iguais ou não.
- __Chi-square ($\chi^2$)__: também é um teste estatístico, só que aplicado para grupos de atributos categóricos para avaliar a probabilidade de correlação ou associação entre eles usando suas distribuições de frequência.

Uma coisa que você deve ter em mente é que __métodos de filtragem NÃO removem multicolinearidade__. Portanto, você também deve se preocupar com a multicolinearidade dos atributos antes de treinar o seu modelo.

## Correlação de Pearson

__Uma desvantagem do coeficiente de correlação de Pearson é que ele é sensível apenas a relações lineares__. Se o relacionamento é não-linear, a correlação de Pearson pode ser próxima a zero mesmo se existir uma correspondência 1:1 entre as duas variáveis. Por exemplo, a correlação entre $x$ e $x^2$ é aproximadamente 0:

In [2]:
x = np.random.uniform(-1, 1, 1000)
print(np.corrcoef(x.T, x**2))

[[ 1.         -0.04496786]
 [-0.04496786  1.        ]]


Você pode conferir alguns exemplos do coeficiente de Pearson para alguns tipos de dados:

<img src='images/selecao_pearson.png'>

Além disso, podemos ver, na figura abaixo, diferentes dados que tem a mesma quantidade de pontos e o mesmo coeficiente de correlação de Pearson, mas gráficos totalmente diferentes. Logo, sempre vale a pena plotar os dados, se possível:

<img src='images/selecao_pearson_plots.png'>

## Mutual Information

Uma opção mais robusta ao coeficiente de Pearson é a __mutual information (MI)__, que mede a dependencia mútua entre as variáveis, geralmente em bits. A mutual information é definida como:

$$I(X, Y) = \sum_{y\in Y} \sum_{x \in X} p(x, y)log \left(\frac{p(x,y)}{p(x)p(y)}\right)$$

Entretanto, ela pode ser inconveniente quando usada diretamente para ranking de atributos por duas razões:

1. __Não é uma métrica e não é normalizada__. Portanto, os valores de MI não podem ser comparados entre dois datasets.
2. __Não é conveniente para variáveis contínuas__. Em geral, as variáveis precisam ser discretizadas para construção dos bins, porém a informação do score pode ser bem sensitiva a seleção dos bins.

## Maximal Information Coefficient

__Maximal Information Coefficient (MIC)__ é uma técnica desenvolvida para tratar as deficiências da _mutual information_. Ela procura por bins ótimos e ainda normaliza o score da _mutual information_ entre $[0, 1]$. Em python, ela pode ser implementada utilizando a biblioteca __minepy__:

```sh
pip install minepy
```

In [3]:
x = np.random.uniform(-1, 1, 1000)

m = MINE()
m.compute_score(x, x**2)
print(m.mic())

1.0000000000000002


Entretanto, __existem algumas críticas ([aqui](http://ie.technion.ac.il/~gorfinm/files/science6.pdf) e [aqui](http://www-stat.stanford.edu/~tibs/reshef/comment.pdf)) a respeito do poder estatístico do MIC, isto é, a capacidade de rejeitar a hipótese nula quando ela é falsa__. Isso pode ou não ser um problema, dependendo do banco e sua ruidosidade.

# Métodos de Empacotamento (Wrapper Methods)

<img src='images/selecao_wrapper_methods.png' width="700">

Os métodos de empacotamento tentam usar um subconjunto de atributos e treinar um modelo com esse subconjunto. __Baseado no resultado do modelo anterior, o algoritmo decide por adicionar ou remover atributos do subconjunto__. Esse problema é essencialmente reduzido a um problema de busca. __Os métodos de empacotamento são geralmente muito caros computacionalmente__.

Os algoritmos mais comums de empacotamento são divididos em 3 grupos:

- __Forward Selection__: método iterativo no qual o algoritmo começa sem nenhum atributo no modelo. Em cada iteração, o algoritmo adiciona o atributo que proporciona o maior ganho de desempenho. O algoritmo termina quando a adição de novos atributos não melhora o desempenho do algoritmo.
- __Backward Elimination__: nesse método, o algoritmo começa com todos os atributos e, a cada iteração, remove o atributo com a menor significância ou que, quando removido, melhora o desempenho do modelo. Esse procedimento é repetido até que nenhuma melhora é observada na remoção dos atributos.
- __Recursive Feature Elimination__: é um algoritmo de otimização guloso cujo objetivo é encontrar o melhor subconjunto de atributos. Repetidamente, o algoritmo cria modelos e mantém o melhor ou o pior atributo a cada iteração. Então, um novo modelo é treinado com os melhores atributos até que os atributos se acabem. Finalmente, o algoritmo faz o ranking dos atributos baseado na ordem das suas eliminações.

Abaixo, veremos alguns dos mais usados métodos de empacotamento

## Stability Selection

[Stability selection](http://stat.ethz.ch/~nicolai/stability.pdf) é considerado um método de seleção de atributos relativamente novo, baseado na subamostragem e combinação de algoritmos de seleção (que podem ser regressores, SVMs ou outros métodos similares). __A ideia principal é aplicar um algoritmo seletor de atributos em diferentes subconjuntos dos dados e com diferentes subconjunto dos atributos__. Após repetir esse processo um certo número de vezes, os resultado da seleção podem ser agregados, por exemplo, contando quantas vezes um atributo foi selecionado como importante quando ele esteve em um subconjunto de atributos. É de se esperar que atributos importantes tenham scores próximos a 100%, uma vez que eles devem sempre ser selecionados quando possível. Atributos fracos, mas ainda relevantes, terão scores não-zero, já que foram selecionados quando atributos importantes não estavam presentes no subconjunto selecionado atual. Atributos irrelavantes, por sua vez, devem ter scores (próximo à) zero, já que eles não devem estar presente nos atributos selecionados.

O sklearn implementa a stability selection nos módulos [randomized lasso](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RandomizedLasso.html) e [randomized logistic regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RandomizedLogisticRegression.html).

In [4]:
boston = load_boston()
x = boston['data']
y = boston['target']
names = boston['feature_names']

rlasso = RandomizedLasso(alpha=0.025, random_state=42)
rlasso.fit(x, y)

print("Atributos ordenados pelo score:")
print(sorted(zip(rlasso.scores_, names), reverse=True))

C:\Users\Arnaldo\Miniconda3\envs\matching\lib\site-packages\sklearn\utils\deprecation.py:57: DeprecationWarning: Class RandomizedLasso is deprecated; The class RandomizedLasso is deprecated in 0.19 and will be removed in 0.21.
  warnings.warn(msg, category=DeprecationWarning)


Atributos ordenados pelo score:
[(1.0, 'RM'), (1.0, 'PTRATIO'), (1.0, 'LSTAT'), (0.61499999999999999, 'B'), (0.58499999999999996, 'CHAS'), (0.40999999999999998, 'TAX'), (0.36499999999999999, 'CRIM'), (0.23999999999999999, 'DIS'), (0.17999999999999999, 'NOX'), (0.115, 'INDUS'), (0.074999999999999997, 'ZN'), (0.029999999999999999, 'AGE'), (0.014999999999999999, 'RAD')]


Como você pode ver, os 3 atributos mais importantes tem score igual a 1.0, ou seja, eles sempre foram escolhidos como atributos úteis. Obviamente, esse resultado pode ser diferente para outros parâmetros de regularização, apesar da implementação do sklearn ser capaz de escolher um bom $\alpha$ automaticamente. O restante dos scores diminui drasticamente depois desses 3 atributos, porém em geral o descarte dos atributos não é tão agressivo quanto no caso da Lasso pura ou Random Forest. __Isso significa que a stability selecion é útil tanto na seleção de atributos pura para reduzir overfitting, quanto na interpretação dos dados__. Em geral, bons atributos terão 0 como coeficiente só por que são similares ou correlacionados no dataset (como na regressão Lasso). __Para seleção de atributos, a stability selection está entre os melhores algoritmos testados em diferentes bancos de dados e configurações__.

## Eliminação recursiva de atributos

__A eliminação recursiva de atributos__ (_recursive feature elimination_, RFE)__ é baseada na ideia de repetidamente construir um modelo__ (por exemplo, SVM ou regressor) __e escolher o melhor ou pior atributo__ (por exemplo, baseado nos coeficientes), __separando esse atributo e repetindo o processo com os demais__. Esse processo é aplicado até que todos os atributs no dataset foram utilizados. __Os atributos são, então, ranqueados de acordo com a ordem de eliminação__. Assim sendo, isso representa um processo guloso de otimização para encontrar o melhor subconjunto de atributos.

__A estabilidade da RFE é bastante dependente do tipo de modelo usado para ranqueamento de atributos em cada iteração__.

O sklearn provê o módulo [RFE](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html) para eliminação recursiva de atributos e [RFECV](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html) para encontrar os rankings e a quantidade ótima de atributos via validação cruzada.

In [5]:
boston = load_boston()
x = boston['data']
y = boston['target']
names = boston['feature_names']

# usa a regressão linear como modelo
lr = LinearRegression()
rfe = RFE(estimator=lr, n_features_to_select=1)
rfe.fit(x, y)

print("Atributos ordenados pelo rank:")
print(sorted(zip(rfe.ranking_, names)))

Atributos ordenados pelo rank:
[(1, 'NOX'), (2, 'RM'), (3, 'CHAS'), (4, 'PTRATIO'), (5, 'DIS'), (6, 'LSTAT'), (7, 'RAD'), (8, 'CRIM'), (9, 'INDUS'), (10, 'ZN'), (11, 'TAX'), (12, 'B'), (13, 'AGE')]


In [6]:
boston = load_boston()
x = boston['data']
y = boston['target']
names = boston['feature_names']

# usa a regressão linear como modelo
lr = LinearRegression()
rfe = RFECV(estimator=lr, step=1, cv=10, n_jobs=-1)
rfe.fit(x, y)

print("Atributos ordenados pelo rank:")
print(sorted(zip(rfe.ranking_, names)))

Atributos ordenados pelo rank:
[(1, 'CHAS'), (1, 'DIS'), (1, 'LSTAT'), (1, 'NOX'), (1, 'PTRATIO'), (1, 'RM'), (2, 'RAD'), (3, 'CRIM'), (4, 'INDUS'), (5, 'ZN'), (6, 'TAX'), (7, 'B'), (8, 'AGE')]


## Boruta

Um dos melhores métodos de empacotamento é o __Boruta__, um método de seleção de todos atributos relevantes. Ao contrário de outros métodos, que tentam encontrar um subconjunto ótimo mínimo que minimizam o erro, __o Boruta tenta encontrar todos os atributos que contém informação útil para predição__.

A ideia por trás do Boruta é simples. Primeiramente, o dataset é duplicado e cada coluna é embaralhada (_shuffle_). Essas colunas são denominadas _shadow features_. Depois, o algoritmo treina um classificador e calcula a importância de cada atributo. Em geral, Random Forest, Gradient Boosted Trees e Extra Trees são utilizados como classificadores. Finalmente, o algoritmo verifica se cada atributo real (não-_shadow_) tem uma importância maior que a melhor _shadow feature_. Em caso afirmativo, nós removemos esses atributos - ele é importante - e o algoritmo continua em outra iteração. Em cada iteração, o algoritmo verifica se o atributo em questão é melhor que _random chance_. Isso é feito simplesmente comparando o número de vez que um atributo foi melhor que as _shadow features_ usando uma distribuição binomial.

<img src='images/boruta-algoritmo.png' width='400'>

Mais detalhes sobre o algoritmo pode ser conferido [aqui](http://danielhomola.com/2015/05/08/borutapy-an-all-relevant-feature-selection-method/).

In [7]:
from third_party.boruta import BorutaPy

x, y = make_regression(n_samples=100, n_features=10, n_informative=3, random_state=42)

rf = RandomForestRegressor(n_estimators=10, max_depth=5, random_state=42)

boruta = BorutaPy(rf, n_estimators='auto', verbose=0, random_state=42) # verbose = 2 para saída completa
boruta.fit(x, y)
print(boruta.support_)
print(boruta.ranking_)

[False  True  True  True False False False False False False]
[6 1 1 1 7 2 5 4 2 4]


# Diferença entre métodos de filtragem e empacotamento

As principais diferenças entre os métodos de filtragem (_filter methods_) e os de empacotamento (_wrapper methods_) são:

| Métodos de Filtragem | Métodos de Empacotamento |
|:--------------------------------------------------------------------------------------------------:|:-------------------------------------------------------------------------------------------------:|
| medem a relevância dos atributos <br>pela suas __"correlações" com a saída__ | medem a utilidade de um subconjunto<br> de atributos __treinando um modelo__ |
| __rápidos__, pois não treinam modelos | __lentos__, pois treinam modelos |
| __métodos estatísticos__<br> para avaliação de um subconjunto | __validação cruzada__<br> para avaliação de um subconjunto |
| podem __NÃO__ encontrar<br> o melhor subcojunto | __muitas vezes__ encontram<br> o melhor subconjunto |
| __menor__ risco de overfitting | __maior__ risco de overfitting |

# Métodos Embarcados (Embedded Methods)

<img src='images/selecao_embedded_methods.png' width="700">

Métodos embarcados combinam as qualidades dos métodos de filtragem e empacotamento. Métodos desse tipo são implementados por algoritmos que têm seu próprio método de seleção de atributos.

Alguns dos mais populares exemplos desses métodos são os algoritmos de regressão __LASSO__ e __RIDGE__, que têm funções de penalização para reduzir overfitting. A diferença básica entre eles é:

- a __regressão LASSO__ utiliza __L1 regularization__ que adiciona uma penalização pelo __valor absoluto__ da magnitude dos coeficientes dos atributos.
- a __regressão RIDGE__ utiliza __L2 regularization__ que adiciona uma penalização pelo __quadrado__ da magnitude dos coeficientes dos atributos.

Outros exemplos de métodos embarcados são __Regularized trees, Memetic algoritm, Random mutinomial logit__, etc.

Nos exemplos abaixo, iremos utilizar coeficientes de modelos de regressão para selecão e interpretação de atributos. Isso baseia-se na ideia de que __quando todos os atributos estão sob a mesma escala, atributos mais importantes devem ter coeficientes mais altos no modelo, enquanto atributos não correlacionados com a saída devem ter coeficientes próximos a zero__. Isso funciona bem inclusive com modelos lineares simples de regressão quando os dados não são muito ruidosos (ou há muito mais dados que atributos) e os atributos são (relativamente) independentes.

## Linear Regression

In [8]:
np.random.seed(0)

x = np.random.normal(0, 1, (5000, 3))
y = x[:, 0] + 2*x[:, 1] + np.random.normal(0, 2, 5000)

lr = LinearRegression()
lr.fit(x, y)
lr.coef_

array([ 0.98422873,  1.99522378, -0.04074316])

Como pode ser observado nesse exemplo, o modelo certamente recupera a estrutura fundamental dos dados muito bem, apesar do ruido significante que colocamos nos dados. __Repare como o coeficiente de $x_2$ é praticamente o dobro de $x_1$, e como o coeficiente do ruído ($x_3$ é praticamente zero)__. Na verdade, esse problema é particularmente simples para um modelo linear: __relacionamento puramente linear entre os atributos e a varíavel de resposta e sem correlação entre os atributos__.

Quando há multiplos atributos (linearmente) correlacionados - como é o caso de muitos dados do mundo real -, o modelo se torna instável, ou seja, pequenas mudanças nos dados pode causar grandes mudanças no coeficientes do modelo, tornando a interpretação do modelo muito difícil, também chamado de __problema da multicolinearidade__.

Vamos ver um exemplo:

In [9]:
size = 100
np.random.seed(5)

x_seed = np.random.normal(0, 1, size)
x1 = x_seed + np.random.normal(0, 0.1, size)
x2 = x_seed + np.random.normal(0, 0.1, size)
x3 = x_seed + np.random.normal(0, 0.1, size)

y = x1 + x2 + x3 + np.random.normal(0, 1, size)
x = np.array([x1, x2, x3]).T

lr = LinearRegression()
lr.fit(x, y)
print(lr.coef_)

[-1.2912413   1.59097473  2.74727029]


Repare que a soma dos coeficientes é $\approx$ 3, então a gente podia esperar que esse modelo funcionasse bem. Por outro lado, se nós interpretarmos os coeficientes pelo seu valor, $x_3$ tem o maior impacto sobre a variável de saída, enquanto $x_1$ tem um valor negativo. Na verdade, todos os atributos são correlacionados quase igualmente à varíavel de saída.

A mesma ideia se aplica a outros modelo lineares, como regressão logística.

## Modelos Reguladores

__Regularização é um método para adicionar restrições ou penalidades a um modelo com o objetivo de prevenir overfitting e melhorar a generalização__. Ao invés de minimizar a função de custo $E(x, y)$, a função de custo torna-se $E(x,y) + \alpha\|w\|$, onde $w$ é o vetor de coeficientes do modelo, $\|\cdot\|$ é tipicamente a norma L1 ou L2 e $\alpha$ é um parâmetro tunável que especifica a força da regularização ($\alpha = 0$ significa "sem regularização"). Em relação aos modelos lineares, os dois métodos de regularização mais utilizados são L1 e L2, também conhecidos como regressões [Lasso](http://en.wikipedia.org/wiki/Least_squares#Lasso_method) e [Ridge](http://en.wikipedia.org/wiki/Tikhonov_regularization), respectivamente, quando aplicados em regressão linear.

### L1 regularization / Lasso

A regularização L1 adiciona uma penalidade da forma $\alpha \sum_{i=1}^n \left|w_i\right|$ à função de perda (normalização L1). Uma vez que cada coeficiente não-zero contribui para a penalidade, isso força fracos atributos a terem coeficientes próximos a zero. Portanto, __a regularização L1 produz soluções esparsas e, consequentemente, efetuando seleção de atributos__. No scikit-learn, a regressão [Lasso](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) é oferecida no módulo de regressão linear, enquanto a [Regressão Logística](http://blog.datadive.net/selecting-good-features-part-ii-linear-models-and-regularization/scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) com penalidade L1 pode ser utilizada para classificação.

In [10]:
boston = load_boston()
scaler = StandardScaler()
x = scaler.fit_transform(boston['data'])
y = boston['target']
names = boston['feature_names']

lasso = Lasso(alpha=0.3, random_state=42)
lasso.fit(x, y)

for c, att in zip(lasso.coef_, names):
    print('{}: {}'.format(att, round(c, 3)))

CRIM: -0.236
ZN: 0.081
INDUS: -0.0
CHAS: 0.54
NOX: -0.7
RM: 2.992
AGE: -0.0
DIS: -1.081
RAD: 0.0
TAX: -0.0
PTRATIO: -1.757
B: 0.631
LSTAT: -3.707


Como a gente pode ver, alguns atributos tem coeficiente 0 (_INDUS, AGE, RAD e TAX_). Se nós aumentar o $\alpha$, a solução seria ainda mais esparsa, isto é, mais e mais atributos teriam coeficiente 0. Em outras palavras, __quanto maior o $\alpha$, mais atributos são zerados (mais esparsa é a solução)__.

Note, entretanto, que __a regularização L1 é instável__ de forma parecida aos modelos não regularizados, ou seja, __os coeficientes podem variar significativamente com pouca mudança nos dados quando existem atributos correlacionados com a saída__. Isso nos leva a regularização L2.

### L2 regularization / Ridge

A __regularização L2__, também chamada de __Ridge Regression__ para a regressão linear) adiciona a penalidade [norma L2](http://en.wikipedia.org/wiki/Norm_%28mathematics%29#Euclidean_norm) ($\alpha\sum_{i=1}^n w_i^2$) a função de perda.

Uma vez que os coeficientes são elevados ao quadrado, ela apresenta um efeito diferente da norma L1, forçando os coeficientes a se "espalharem" mais igualmente. __Para atributos correlacionados, isso significa que eles tendem a ter coeficientes parecidos__.

Voltando ao exemplo de $y = x_1 + x_2$, onde $x_1$ e $x_2$ são altamente correlacionados. 
- para a norma L1, a penalidade é a mesma independentemente se o modelo aprendido é $y = 1*x_1 + 1*x_2$ ou $y = 2*x_1 + 0*x_2$. Em ambos os casos, a penalidade é $2\alpha$ (soma dos coeficientes). 
- Para a norma L2, entretanto, a penalidade do primeiro modelo é $1^2 + 1^2 = 2\alpha$, enquanto que para o segundo modelo é $2^2 + 0^2 = 4\alpha$.

__O efeito disso é que os modelos são muito mais estáveis quando utilizamos a regularização L2__. Os coeficientes não "flutuam" em pequenas mudanças nos dados como acontece nos modelos não regularizados ou modelos L1. Portanto, __apesar de a regualarização L2 não fazer seleção de atributos como a L1 faz, ela é mais útil para _interpretação_ de atributos: um atributo preditivo vai ter um coeficiente diferente de zero, que não é o caso da L1__.

Vamos ver o exemplo dos 3 atributos correlacionados novamente, treinando um modelo diferentes para 10 dados aleatórios, para mostramos a estabilidade da regressão L2:

In [11]:
size = 100

for i in range(10):
    print("Random seed:", i)
    np.random.seed(seed=i)
    x_seed = np.random.normal(0, 1, size)
    x1 = x_seed + np.random.normal(0, 0.1, size)
    x2 = x_seed + np.random.normal(0, 0.1, size)    
    x3 = x_seed + np.random.normal(0, 0.1, size)    
    y = x1 + x2 + x3 + np.random.normal(0, 1, size)
    x = np.array([x1, x2, x3]).T
    
    lr = LinearRegression()
    lr.fit(x, y)
    print("Linear model:", lr.coef_)
    
    ridge = Ridge(alpha=10)
    ridge = ridge.fit(x, y)
    print("Ridge model:", ridge.coef_)
    print()

Random seed: 0
Linear model: [ 0.7284403   2.30926001 -0.08219169]
Ridge model: [ 0.93832131  1.05887277  0.87652644]

Random seed: 1
Linear model: [ 1.15181561  2.36579916 -0.59900864]
Ridge model: [ 0.98409577  1.06792673  0.75855367]

Random seed: 2
Linear model: [ 0.69734749  0.32155864  2.08590886]
Ridge model: [ 0.97159124  0.94256202  1.08539406]

Random seed: 3
Linear model: [ 0.28735446  1.25386129  1.49054726]
Ridge model: [ 0.91891806  1.00474386  1.03276594]

Random seed: 4
Linear model: [ 0.18726691  0.77214206  2.1894915 ]
Ridge model: [ 0.96401621  0.98152524  1.0983599 ]

Random seed: 5
Linear model: [-1.2912413   1.59097473  2.74727029]
Ridge model: [ 0.75819864  1.01085804  1.1390417 ]

Random seed: 6
Linear model: [ 1.19909595 -0.0306915   1.91454912]
Ridge model: [ 1.01616507  0.89032238  1.0907386 ]

Random seed: 7
Linear model: [ 1.47386769  1.76236014 -0.15057274]
Ridge model: [ 1.0179376   1.03865514  0.90082373]

Random seed: 8
Linear model: [ 0.0840547   1.879

Como você pode observar, os coeficientes variam bem mais no caso da regressão linear, dependendo dos dados gerados. __No modelo que usa regularização L2, entrentato, os coeficientes são bem estáveis e refletem aproximadamente como os dados são gerados (todos os coeficientes são próximos a 1)__.

Em suma, podemos dizer que:
- a __regressão Lasso__ produz __soluções esparsas__ e é bastante útil em __selecionar um bom subconjunto de atributos__ para melhorar a performance do modelo.
- a __regressão Ridge__ pode ser usada para __interpretação de atributos__ devido sua __estabilidade__ e o fato que atributos úteis tendem a ter coeficientes não-zero.

## Random Forest

Random forest provêm dois métodos para seleção de atributos: __mean decrease impurity__ e __mean decrease accuracy__.

### Mean Decrease Impurity

A medida que uma árvore utiliza para calcular a melhor condição de divisão de um nó é chamada impureza. Para classificação, é tipicamente a [giny impurity](http://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity) ou [ganho de informação/entropia](http://en.wikipedia.org/wiki/Information_gain_in_decision_trees). No caso da regressão, a mais comum é a [variância](http://en.wikipedia.org/wiki/Variance). Portanto, quando uma árvore está em processo de treinamento, ela calcula quanto cada atributo diminui a impureza ponderada. Para uma floresta, calculamos a média da impureza que diminui de cada atributo e ranqueamos de acordo com essa medida.

In [12]:
boston = load_boston()
x = boston['data']
y = boston['target']
names = boston['feature_names']

rf = RandomForestRegressor(random_state=42)
rf.fit(x, y)

ranking = sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), names), reverse=True)
print("Atributos ordenados pelo score:")
for score, att in ranking:
    print('{}: {}'.format(att, score))

Atributos ordenados pelo score:
LSTAT: 0.4084
RM: 0.397
DIS: 0.0642
CRIM: 0.0444
NOX: 0.0225
PTRATIO: 0.0171
AGE: 0.014
TAX: 0.0124
B: 0.0092
INDUS: 0.0075
RAD: 0.002
ZN: 0.0008
CHAS: 0.0004


Há algumas coisas a se ter em mente quando usar métodos de ranqueamento baseado em impureza:
- __a seleção de atributos baseada na redução de impureza é enviesada a preferir variáveis com mais categorias__ (ver discussões [nesse link](http://link.springer.com/article/10.1186%2F1471-2105-8-25)).
- __quando o dataset tem dois ou mais atributos correlacionados__, então, do ponto de vista do modelo, __apenas um desses atributos precisa ser usado como preditor__, sem haver preferência sobre algum deles. Porém, uma vez que um deles é utilizado, a importância dos outros é significantemente reduzida já que a impureza removida por eles já foi removida pelo primeiro atributo. Como consequência, eles sempre terão uma importância reduzida menor. __Isso não é um problema quando estamos fazendo seleção de atributo para reduzir overfitting__, já que faz sentido remover atributos que estão provavelmente duplicados por outros atributos. Porém, __ao interpretar os dados, isso pode levar a conclusões incorretas que um atributo é importante para predição enquanto outros no mesmo grupo não são__. Na verdade, esses atributos são tão próximos em termos de relacionamento com a variável resposta.

O efeito desse fenômeno é de tal forma reduzido graças a seleção de atributos automática em cada nó criado, mas em geral os efeitos não são reduzidos completamente. No exemplo a seguir, nós temos 3 variáveis correlacionadas $x_1$, $x_2$ e $x_3$ e a saída sendo somente a soma dos 3 atributos

In [13]:
seed = 10
np.random.seed(seed)

size = 10000
x_seed = np.random.normal(0, 1, size)
x0 = x_seed + np.random.normal(0, 0.1, size)
x1 = x_seed + np.random.normal(0, 0.1, size)
x2 = x_seed + np.random.normal(0, 0.1, size)
x = np.array([x0, x1, x2]).T
y = x0 + x1 + x2

rf = RandomForestRegressor(n_estimators=20, max_features=2, random_state=seed)
rf.fit(x, y)
print("Scores para x0, x1 e x2:", rf.feature_importances_)

Scores para x0, x1 e x2: [ 0.35475549  0.53280751  0.112437  ]


Repare que a importancia de $x_1$ aparenta ser quase 5x maior que $x_2$, enquanto que a importância de todos os atributos é praticamente igual. Isso acontece mesmo utilizando 20 árvores, seleção de atributos randômica, praticamente sem ruído nos dados e um dataset grande o suficiente (10000 amostras).

__Além disso, é importante frisar que esse efeito não é exclusivo de Random Forests, mas comum a maioria dos modelos com seleção de atributos__.

### Mean Decrease Accuracy

Outro método popular de seleção de atributos é calcular diretamente o impacto de cada atributo na acurácia do modelo. A ideia geral é permutar os valores de cada atributo e medir o quanto essa permutação diminui a acurácia. Claramente, a permutação de variáveis pouco importantes não surtirá efeito na acurácia do modelo, enquanto variáveis mais importantes devem diminuir a acurácia significantemente.

In [14]:
boston = load_boston()
x = boston['data']
y = boston['target']
names = boston['feature_names']

rf = RandomForestRegressor(random_state=42)
scores = defaultdict(list)

shuffle = ShuffleSplit(n_splits=100, test_size=0.3, random_state=42)
for train_idx, test_idx in shuffle.split(x):
    x_train, x_test = x[train_idx], x[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    rf.fit(x_train, y_train)
    
    acc = r2_score(y_test, rf.predict(x_test))
    for i in range(x.shape[1]):
        x_t = x_test.copy()
        np.random.shuffle(x_t[:, i])
        shuff_acc = r2_score(y_test, rf.predict(x_t))
        scores[names[i]].append((acc-shuff_acc)/acc)

print("Atributos ordenados pelo score:")
print(sorted( [(np.mean(score), feat) for feat, score in scores.items()] , reverse=True))

Atributos ordenados pelo score:
[(0.72885675528428595, 'LSTAT'), (0.5362736330186163, 'RM'), (0.077228114994336738, 'DIS'), (0.039458747084910678, 'CRIM'), (0.036445334276048361, 'NOX'), (0.023260283653553048, 'PTRATIO'), (0.017648160496050759, 'TAX'), (0.011202359139659534, 'AGE'), (0.0056505793730686127, 'INDUS'), (0.0053741923956472102, 'B'), (0.0037062592057342412, 'RAD'), (0.00059175625463918303, 'ZN'), (3.1996482898812873e-05, 'CHAS')]


Nesse exemplo, __LSTAT__ e __RM__ são os dois atributos com o maior impacto de performance: permutá-los diminui a acurácia em aproximadamente 73% e 54%, respectivamente. Lembre-se, porém, que essas medidas são feitas somente após o modelo ter sido treinado (e depende) de todos esses atributos. __Isso não significa que, se treinarmos o modelo sem um desses atributos, o desempenho do modelo vai cair por esse valor, uma vez que atributos correlatos podem estar sendo usados__.

Finalizando: __Random Forest é um método muito bom para ranking de atributos, mas deve-se tomar cuidado ao interpretar os coeficientes. Com atributos correlacionados, atributos fortes podem terminar com baixos scores e o método pode ser enviesado por variáveis com muitas categorias__. Porém, com tudo isso mente, vale a pena testá-las em seus dados.

# Exemplo prático

Para finalizar esse notebook, vamos aplicar a maioria dos métodos de seleção de atributos que vimos para compará-los. O dataset que vamos utilizar é chamado de __Friedman \#1 regression dataset__ ([fonte](ftp://ftp.uic.edu/pub/depts/econ/hhstokes/e538/Friedman_mars_1991.pdf)). Os dados são gerados de acordo com a seguinte fórmula:

$$y = 10sin(\pi x_1 x_2) + 20(x_3 – 0.5)^2 + 10x_4 + 5x_5 +\epsilon$$

onde, $x_1$ a $x_5$ são amostrados de uma distribuição uniforme e $\epsilon$ é um ruído com distribuição normal $N(0,1)$. Além disso, o dataset original tem cinco atributos de ruído $x_6,\dots,x_{10}$ independentes da variável de resposta. Nós vamos aumentar o número de atributos com mais 4 variáveis $x_{11}, \dots, x_{14}$ que são fortemente correlacionados com $x_1,\dots,x_4$, respectivamente, geradas através de:

$$f(x) = x + N(0, 0.01)$$

Isso produz um coeficiente de correlação maior que $0.999$ entre as variáveis. __Nosso objetivo com esse exemplo é mostrar como diferentes métodos de ranqueamento lidam com as correlações nos dados__.

Nós vamos também normalizar os scores de ranqueamento para que fiquem entre 0 (para o atributo menos importante) e 1 (para o atributo mais importante). No caso da eliminação recursiva de atributos, os 5 atributos mais importantes vão receber score 1, e o restante dos atributos terá o score normalizado entre 0 e 1 de acordo com sua posição.

In [15]:
seed = 0
np.random.seed(seed)

size = 750
x = np.random.uniform(0, 1, (size, 14))
y = (10 * np.sin(np.pi*x[:,0]*x[:,1]) + 20*(x[:,2] - .5)**2 + 10*x[:,3] + 5*x[:,4] + np.random.normal(0,1))

# adiciona 3 variáveis correlacionados com x1-x3
x[:,10:] = x[:,:4] + np.random.normal(0, .025, (size,4))
 
names = ["x%s" % i for i in range(1,15)] 
ranks = {}

def rank_to_dict(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = [round(x, 2) for x in ranks]
    return ranks

lr = LinearRegression(normalize=True)
lr.fit(x, y)
ranks["Lin. Reg."] = rank_to_dict(np.abs(lr.coef_), names)
 
ridge = Ridge(alpha=7)
ridge.fit(x, y)
ranks["Ridge"] = rank_to_dict(np.abs(ridge.coef_), names)
 
lasso = Lasso(alpha=.05)
lasso.fit(x, y)
ranks["Lasso"] = rank_to_dict(np.abs(lasso.coef_), names)
 
rlasso = RandomizedLasso(alpha=0.04, random_state=seed)
rlasso.fit(x, y)
ranks["Stability"] = rank_to_dict(np.abs(rlasso.scores_), names)
 
rfe = RFE(lr, n_features_to_select=5)
rfe.fit(x, y)
ranks["RFE"] = rank_to_dict(list(map(float, rfe.ranking_)), names, order=-1)
 
rf = RandomForestRegressor(random_state=seed)
rf.fit(x, y)
ranks["RF"] = rank_to_dict(rf.feature_importances_, names)
 
f, pval  = f_regression(x, y, center=True)
ranks["Corr."] = rank_to_dict(f, names)
 
mine = MINE()
mic_scores = []
for i in range(x.shape[1]):
    mine.compute_score(x[:,i], y)
    m = mine.mic()
    mic_scores.append(m)

ranks["MIC"] = rank_to_dict(mic_scores, names)

df = pd.DataFrame.from_dict(ranks)
df = df.rename(index = dict(zip(range(14), names)))
df

C:\Users\Arnaldo\Miniconda3\envs\matching\lib\site-packages\sklearn\utils\deprecation.py:57: DeprecationWarning: Class RandomizedLasso is deprecated; The class RandomizedLasso is deprecated in 0.19 and will be removed in 0.21.
  warnings.warn(msg, category=DeprecationWarning)


,Corr.,Lasso,Lin. Reg.,MIC,RF,RFE,Ridge,Stability
x1,0.30,0.79,1.00,0.39,0.41,1.00,0.77,0.77
x2,0.44,0.83,0.56,0.61,0.50,1.00,0.75,0.73
x3,0.00,0.00,0.50,0.34,0.07,1.00,0.05,0.00
x4,1.00,1.00,0.57,1.00,0.19,1.00,1.00,1.00
x5,0.10,0.51,0.27,0.20,0.20,0.78,0.88,0.64
x6,0.00,0.00,0.02,0.00,0.01,0.44,0.05,0.00
x7,0.01,0.00,0.00,0.07,0.01,0.00,0.01,0.00
x8,0.02,0.00,0.03,0.05,0.00,0.56,0.09,0.00
x9,0.01,0.00,0.00,0.09,0.02,0.11,0.00,0.00
x10,0.00,0.00,0.01,0.04,0.00,0.33,0.01,0.00


Os resultados acima apresentam algumas características interessantes:
- __Linear Regression__: como cada atributo é avaliado independentemente, os scores para os atributos $x_1,\dots,x_4$ são muito similares a $x_{11},\dots,x_{14}$, enquanto os atributos ruido $x_5,\dots,x_{10}$ são corretamente identificados para ter quase nenhuma relação com a variável de resposta. Não é possível identificar qualquer relacionamento entre $x_3$ e $y$, já que esse relacionamento é quadrático (de fato, isso se aplica a quase todos os métodos com exceção do MIC). Também é possível observar que enquanto o método é capaz de medir o relacionamento linear entre cada atributo e a variável resposta, ele não é ótimo para selecionar os melhores atributos para melhorar a generalização do modelo, já que os atributos mais importantes serão essencialmente escolhidos duas vezes.
- __Lasso__: esse método foi capaz de selecionar os melhores atributos, enquant força outros atributos a serem próximos a zero. Ele é claramente útil quando a intenção é reduzir o número de atributos, mas não necessariamente interpretar os dados (já que ele nos faz acreditar que os atributos $x_{11},\dots,x_{13}$ não tem uma relação forte com a variável resposta).
- __MIC__: é similar ao coeficiente de correlação em relação a tratar todos os atributos "igualmente". Além disso, esse método é capaz de encontrar o relacionamento não-linear entre $x_3$ e a variável de resposta.
- __Random Forest__: a métrica de ranqueamento baseada na impureza da RF é tipicamente agressiva no sentido que há uma queda drástica nos scores depois de alguns atributos no topo do ranking. Isso pode ser visto no exemplo acima, onde o 4º atributo já é praticamente 3x menor que o primeiro (enquanto que para outros métodos de ranqueamento, essa queda não é tão agressiva).
- __Ridge Regression__: esse método força os coeficientes da regressão a se espalharem similarmente entre as variáveis correlacionadas. Isso é claramente visível no exemplo onde $x_{11},\dots,x_{14}$ são próximos a $x_1,\dots,x_4$ em termos de scores.
- __Stability Selection__: é geralmente capaz de fazer um bom compromisso entre interpretação de dados e seleção de melhores atributos para melhoramento do modelo. Isso é bem ilustrado no examplo. Assim como Lasso, esse método é capaz de identificar os melhores atributos ($x_1, x_2, x_4, x_5$). Da mesma forma, as variáveis correlacionadas $x_{11}, x_{12}$ e $x_{14}$ também tem um alto score, mostrando sua relação com a resposta.

# Conclusões

A seleção de atributos pode ser incrivelmente útil em uma ampla gama de aplicações em machine learning e data mining. __É importante ter em mente o por quê você está fazendo a seleção de atributos e entender qual método funciona melhor para esse propósito__. Quando selecionar os melhores atributos para melhorar o desempenho do modelo, __é fácil verificar se um método particular funciona bem em relação a outras alternativas simplesmente efetuando uma validação cruzada. Porém, não é tão simples quando utilizamos o ranqueamento para interpretação dos dados, onde a estabilidade do método de rankeamento é crucial e, se o método não possui essa propriedade (como a regressão Lasso), podemos facilmente tirar conclusões incorretas__. O que pode ser feito é subamostrar os dados e rodar algoritmos de seleção nos subconjuntos. Se os resultados são consistentes nos subconjuntos, é relativamente seguro confiar na estabilidade do método nesses dados particulares e, portanto, fácil de interpretar os dados em termos de ranqueamento.

# Referências

- [Feature Selection - Wikipedia](https://en.wikipedia.org/wiki/Feature_selection)
- [Selecting good features – Part I: univariate selection](http://blog.datadive.net/selecting-good-features-part-i-univariate-selection/)
- [Selecting good features – Part II: linear models and regularization](http://blog.datadive.net/selecting-good-features-part-ii-linear-models-and-regularization/)
- [Selecting good features – Part III: random forests](http://blog.datadive.net/selecting-good-features-part-iii-random-forests/)
- [Selecting good features – Part IV: stability selection, RFE and everything side by side](http://blog.datadive.net/selecting-good-features-part-iv-stability-selection-rfe-and-everything-side-by-side/)
- [Introduction to Feature Selection methods with an example (or how to select the right variables?)](https://www.analyticsvidhya.com/blog/2016/12/introduction-to-feature-selection-methods-with-an-example-or-how-to-select-the-right-variables/)
- [BorutaPy](http://danielhomola.com/2015/05/08/borutapy-an-all-relevant-feature-selection-method/)